In [1]:
import urllib2
import tensorflow as tf
import numpy as np

In [2]:
Xdata = np.load("hw2-train-data.npy")
Ydata = np.load("hw2-train-labels.npy").reshape(-1,1)
print("data shape = ", Xdata.shape)
print("labels shape = ", Ydata.shape)

n = 50000
Xtrn,Ytrn = (Xdata[:n]-np.mean(Xdata[:n]))/np.std(Xdata[:n]),np.squeeze(Ydata[:n]) # half of the points for training
print(np.squeeze(Ytrn))
ntrn,dim = Xtrn.shape
Xval,Yval = (Xdata[n:]-np.mean(Xdata[n:]))/np.std(Xdata[n:]),np.squeeze(Ydata[n:]) # rest for model evaluation


('data shape = ', (57500, 768))
('labels shape = ', (57500, 1))
[5 0 4 ..., 8 4 8]


In [3]:
n_inputs = 768
n_hidden1 = 100
n_hidden2 = 200
n_outputs = 10

momentum = 0.9
learning_rate_1 = 0.1
steps = 10

learning_rate_2 = 0.001
epochs = 10
batch = 25 

In [21]:
# 3), 4)

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.shape[1])
        stddev = 2/np.sqrt(n_inputs)
        init = tf.random_normal([n_inputs, n_neurons],stddev = stddev)
        
        W = tf.Variable(init,dtype=tf.float32, name="weights")
        b = tf.Variable(tf.random_normal([1, n_neurons]), dtype=tf.float32, name="biases")
        z = tf.matmul(X, W) + b
        max_ = tf.reduce_max(z, axis=1, keep_dims=True)
        if activation == "relu":
            z = tf.maximum(z,tf.constant(0, dtype=z.dtype))
        elif activation == "softmax":
            z = tf.exp(z-max_)/(tf.reduce_sum(tf.exp(z-max_), axis=1, keep_dims=True))
        else:
            pass
        return z,W
    
def mlp(X,y,learning_rate,momentum = 0.0,layers = [100,200,10], activations = ["relu","relu","softmax"], optimizer = "None"):        
    
    ntrn = tf.cast(tf.shape(y)[0], tf.float32)
    weights = []
    temp = []
    updates = []
    
    with tf.name_scope("MLP"):
        for i,el in enumerate(layers):
            X,w = neuron_layer(X, el, "hidden_"+str(i),activations[i])
            weights.append(w)
            temp.append(w)
            updates.append(w)
            
    with tf.name_scope("xloss"):
        xentropy = - (1/ntrn) * tf.reduce_sum(tf.multiply(tf.log(1e-20 + X),tf.one_hot(y,10, dtype=tf.float32)))
    with tf.name_scope("eval"):    
        accuracy = (1/ntrn) * tf.reduce_sum(tf.cast(tf.equal(tf.argmax(X, axis = 1,output_type=tf.int32),y),tf.float32))
    
    
    if optimizer == "adam":
        train = tf.train.AdamOptimizer(learning_rate = learning_rate, beta1 = momentum).minimize(xentropy)
    elif optimizer == "gradient":
        train = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(xentropy)
    else:
        with tf.name_scope("train"):
            grads = tf.gradients(xentropy, weights)   

        for i,w in enumerate(weights):
            temp[i] = w
            updates[i] = w.assign(w - learning_rate*grads[i] + momentum*(w-temp[i]))
            
        return xentropy,accuracy, X, updates

    return xentropy,accuracy,X, train


In [13]:
# 5)
# No, one cannot train by maximizing accuracy, because e.g. the predictions 0.3 0.4 0.3, 0.1 0.8 0.1 would be classified with the same reward, though the second one is much more useful for the training

tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")
loss, acc, softmax, up = mlp(X,y,0.1)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init) # init variables
    for i in range(steps):
        loss_train, acc_train, _, _, _ = sess.run([loss,acc,up[0],up[1],up[2]], feed_dict={X: Xtrn, y:Ytrn})
        loss_val, acc_val = sess.run([loss, acc], feed_dict={X: Xval, y:Yval})
        print("step", i, "loss(train)", loss_train, "loss(val)", loss_val, "accuracy(train)", acc_train, "accuracy(val)", acc_val)


('step', 0, 'loss(train)', 6.5835667, 'loss(val)', 5.9079199, 'accuracy(train)', 0.10614, 'accuracy(val)', 0.22)
('step', 1, 'loss(train)', 6.0507092, 'loss(val)', 5.0385585, 'accuracy(train)', 0.21023999, 'accuracy(val)', 0.37626666)
('step', 2, 'loss(train)', 5.0508895, 'loss(val)', 3.4972706, 'accuracy(train)', 0.36095998, 'accuracy(val)', 0.45000002)
('step', 3, 'loss(train)', 3.4920006, 'loss(val)', 2.3444526, 'accuracy(train)', 0.4418, 'accuracy(val)', 0.54826665)
('step', 4, 'loss(train)', 2.4913259, 'loss(val)', 1.6914626, 'accuracy(train)', 0.52561998, 'accuracy(val)', 0.61973333)
('step', 5, 'loss(train)', 1.730511, 'loss(val)', 1.0860339, 'accuracy(train)', 0.61289996, 'accuracy(val)', 0.70200002)
('step', 6, 'loss(train)', 1.1655648, 'loss(val)', 1.0096701, 'accuracy(train)', 0.68589997, 'accuracy(val)', 0.71013331)
('step', 7, 'loss(train)', 1.0768079, 'loss(val)', 0.95543569, 'accuracy(train)', 0.69738001, 'accuracy(val)', 0.73720002)
('step', 8, 'loss(train)', 1.0282131,

In [14]:
#6)

def stoch(batch = batch, epochs = epochs, learning_rate = learning_rate_2, momentum = 0.0, only_last = False, optimizer = "None", test = None):
    print("batch =",batch,"learning_rate =",learning_rate,"momentum = ", momentum)
    tf.reset_default_graph()

    X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
    y = tf.placeholder(tf.int32, shape=(None), name="y")
    loss, acc, softmax, up = mlp(X,y,learning_rate, optimizer = optimizer)
    perms = np.array_split(np.random.permutation(len(Xtrn)),len(Xtrn)/batch)
    init = tf.global_variables_initializer()

    with tf.Session() as sess:
        sess.run(init) # init variables
        for i in range(epochs):
            for perm in perms:
                if optimizer not in ["adam","gradient"]:
                    loss_train, acc_train, _, _, _ = sess.run([loss,acc,up[0],up[1],up[2]], feed_dict={X: Xtrn[perm], y:Ytrn[perm]})
                else:
                    loss_train, acc_train, _ = sess.run([loss,acc,up], feed_dict={X: Xtrn[perm], y:Ytrn[perm]})
            loss_val, acc_val = sess.run([loss, acc], feed_dict={X: Xval, y:Yval})
            if only_last == False:
                print("step", i, "loss(train)", loss_train, "loss(val)", loss_val, "accuracy(train)", acc_train, "accuracy(val)", acc_val)
        if only_last == True:
            print("step", i, "loss(train)", loss_train, "loss(val)", loss_val, "accuracy(train)", acc_train, "accuracy(val)", acc_val)
        if test is not None:
            return sess.run(softmax, feed_dict={X: test})

In [23]:
#6) The criteria are met (except for the acc_{val} - almost met)
stoch(batch = 25, learning_rate =  0.001)

('batch =', 25, 'learning_rate =', 0.001, 'momentum = ', 0.0)
('step', 0, 'loss(train)', 0.32131037, 'loss(val)', 0.5573104, 'accuracy(train)', 0.88, 'accuracy(val)', 0.8452)
('step', 1, 'loss(train)', 0.21144097, 'loss(val)', 0.42568603, 'accuracy(train)', 0.91999996, 'accuracy(val)', 0.88066667)
('step', 2, 'loss(train)', 0.16288024, 'loss(val)', 0.36696133, 'accuracy(train)', 0.95999998, 'accuracy(val)', 0.89693332)
('step', 3, 'loss(train)', 0.13273111, 'loss(val)', 0.33148062, 'accuracy(train)', 0.95999998, 'accuracy(val)', 0.90680003)
('step', 4, 'loss(train)', 0.11092057, 'loss(val)', 0.30658606, 'accuracy(train)', 0.95999998, 'accuracy(val)', 0.91439998)
('step', 5, 'loss(train)', 0.092542157, 'loss(val)', 0.28789929, 'accuracy(train)', 0.95999998, 'accuracy(val)', 0.92093337)
('step', 6, 'loss(train)', 0.077106677, 'loss(val)', 0.27303511, 'accuracy(train)', 0.95999998, 'accuracy(val)', 0.92586666)
('step', 7, 'loss(train)', 0.06567318, 'loss(val)', 0.26063758, 'accuracy(train

In [100]:
#7) Model selection

[stoch(batch = batch_, learning_rate = learning_rate_, momentum = 0, only_last = True, optimizer = "adam") for batch_ in np.arange(batch-10,batch+10) for learning_rate_ in np.linspace(learning_rate_2,learning_rate_2*100, 20)]

('batch =', 15, 'learning_rate =', 0.001, 'momentum = ', 0)
<unknown>
('step', 9, 'loss(train)', 0.019795652, 'loss(val)', 0.22395542, 'accuracy(train)', 1.0, 'accuracy(val)', 0.94106668)
('batch =', 15, 'learning_rate =', 0.0062105263157894736, 'momentum = ', 0)
<unknown>
('step', 9, 'loss(train)', 0.017430225, 'loss(val)', 0.16475403, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96186668)
('batch =', 15, 'learning_rate =', 0.011421052631578946, 'momentum = ', 0)
<unknown>
('step', 9, 'loss(train)', 0.010429282, 'loss(val)', 0.17766048, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96293336)
('batch =', 15, 'learning_rate =', 0.016631578947368421, 'momentum = ', 0)
<unknown>
('step', 9, 'loss(train)', 0.020016417, 'loss(val)', 0.1728303, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96880001)
('batch =', 15, 'learning_rate =', 0.021842105263157895, 'momentum = ', 0)
<unknown>
('step', 9, 'loss(train)', 0.0090459455, 'loss(val)', 0.18049169, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96680003)

('step', 9, 'loss(train)', 0.020228554, 'loss(val)', 0.16401224, 'accuracy(train)', 1.0, 'accuracy(val)', 0.95973337)
('batch =', 17, 'learning_rate =', 0.011421052631578946, 'momentum = ', 0)
<unknown>
('step', 9, 'loss(train)', 0.0073581198, 'loss(val)', 0.16462366, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96399999)
('batch =', 17, 'learning_rate =', 0.016631578947368421, 'momentum = ', 0)
<unknown>
('step', 9, 'loss(train)', 0.0010305832, 'loss(val)', 0.16343728, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96493334)
('batch =', 17, 'learning_rate =', 0.021842105263157895, 'momentum = ', 0)
<unknown>
('step', 9, 'loss(train)', 0.00088559347, 'loss(val)', 0.19571766, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96586668)
('batch =', 17, 'learning_rate =', 0.02705263157894737, 'momentum = ', 0)
<unknown>
('step', 9, 'loss(train)', 0.0026118553, 'loss(val)', 0.18161215, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96693337)
('batch =', 17, 'learning_rate =', 0.032263157894736841, 'momentum

('step', 9, 'loss(train)', 0.017332558, 'loss(val)', 0.1716848, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96386665)
('batch =', 19, 'learning_rate =', 0.016631578947368421, 'momentum = ', 0)
<unknown>
('step', 9, 'loss(train)', 0.00021358275, 'loss(val)', 0.15671687, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96693337)
('batch =', 19, 'learning_rate =', 0.021842105263157895, 'momentum = ', 0)
<unknown>
('step', 9, 'loss(train)', 0.0057748747, 'loss(val)', 0.15558983, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96960002)
('batch =', 19, 'learning_rate =', 0.02705263157894737, 'momentum = ', 0)
<unknown>
('step', 9, 'loss(train)', 0.00099837384, 'loss(val)', 0.17501289, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96693337)
('batch =', 19, 'learning_rate =', 0.032263157894736841, 'momentum = ', 0)
<unknown>
('step', 9, 'loss(train)', 0.00094543886, 'loss(val)', 0.17673333, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96746665)
('batch =', 19, 'learning_rate =', 0.037473684210526319, 'momentu

('step', 9, 'loss(train)', 0.0056138216, 'loss(val)', 0.17488949, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96453333)
('batch =', 21, 'learning_rate =', 0.021842105263157895, 'momentum = ', 0)
<unknown>
('step', 9, 'loss(train)', 0.0017171106, 'loss(val)', 0.16388206, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96693337)
('batch =', 21, 'learning_rate =', 0.02705263157894737, 'momentum = ', 0)
<unknown>
('step', 9, 'loss(train)', 0.018820409, 'loss(val)', 0.18530455, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96533334)
('batch =', 21, 'learning_rate =', 0.032263157894736841, 'momentum = ', 0)
<unknown>
('step', 9, 'loss(train)', 0.010936894, 'loss(val)', 0.1581476, 'accuracy(train)', 1.0, 'accuracy(val)', 0.97079998)
('batch =', 21, 'learning_rate =', 0.037473684210526319, 'momentum = ', 0)
<unknown>
('step', 9, 'loss(train)', 0.0015374839, 'loss(val)', 0.17814283, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96946669)
('batch =', 21, 'learning_rate =', 0.04268421052631579, 'momentum = '

('step', 9, 'loss(train)', 0.0032989406, 'loss(val)', 0.17624249, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96746665)
('batch =', 23, 'learning_rate =', 0.02705263157894737, 'momentum = ', 0)
<unknown>
('step', 9, 'loss(train)', 0.0014255026, 'loss(val)', 0.19101055, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96386665)
('batch =', 23, 'learning_rate =', 0.032263157894736841, 'momentum = ', 0)
<unknown>
('step', 9, 'loss(train)', 0.003583129, 'loss(val)', 0.16729495, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96413332)
('batch =', 23, 'learning_rate =', 0.037473684210526319, 'momentum = ', 0)
<unknown>
('step', 9, 'loss(train)', 0.0034668355, 'loss(val)', 0.18151477, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96586668)
('batch =', 23, 'learning_rate =', 0.04268421052631579, 'momentum = ', 0)
<unknown>
('step', 9, 'loss(train)', 0.00067274761, 'loss(val)', 0.17566492, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96826667)
('batch =', 23, 'learning_rate =', 0.047894736842105261, 'momentum 

('step', 9, 'loss(train)', 0.011082649, 'loss(val)', 0.16229163, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96759999)
('batch =', 25, 'learning_rate =', 0.032263157894736841, 'momentum = ', 0)
<unknown>
('step', 9, 'loss(train)', 0.0011839323, 'loss(val)', 0.16299695, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96826667)
('batch =', 25, 'learning_rate =', 0.037473684210526319, 'momentum = ', 0)
<unknown>
('step', 9, 'loss(train)', 0.00025300856, 'loss(val)', 0.16957194, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96853334)
('batch =', 25, 'learning_rate =', 0.04268421052631579, 'momentum = ', 0)
<unknown>
('step', 9, 'loss(train)', 0.00050640455, 'loss(val)', 0.16989775, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96880001)
('batch =', 25, 'learning_rate =', 0.047894736842105261, 'momentum = ', 0)
<unknown>
('step', 9, 'loss(train)', 0.0012554175, 'loss(val)', 0.18960515, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96706671)
('batch =', 25, 'learning_rate =', 0.053105263157894739, 'momentu

('step', 9, 'loss(train)', 0.0010714697, 'loss(val)', 0.18097182, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96533334)
('batch =', 27, 'learning_rate =', 0.037473684210526319, 'momentum = ', 0)
<unknown>
('step', 9, 'loss(train)', 0.0028924181, 'loss(val)', 0.16737214, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96786666)
('batch =', 27, 'learning_rate =', 0.04268421052631579, 'momentum = ', 0)
<unknown>


KeyboardInterrupt: 

In [73]:
# 8), 9) both adam and gradient optimizers are implemented, the architecture selection and the selected model are shown before. Some parameters may have changed in the original function.
    
Xtst = np.load("hw2-test-data.npy")
yTst = stoch(batch = 22, learning_rate =  0.047894, momentum = 0.0, only_last = False, optimizer = "None", test = Xtst)
yTst = np.argmax(yTst,axis=1)
np.savetxt('hw2-test-labels.txt',yTst,fmt='%d')

('batch =', 22, 'learning_rate =', 0.047894, 'momentum = ', 0.0)
('step', 0, 'loss(train)', 0.18519944, 'loss(val)', 0.19125611, 'accuracy(train)', 0.90909094, 'accuracy(val)', 0.94573337)
('step', 1, 'loss(train)', 0.08629781, 'loss(val)', 0.13765965, 'accuracy(train)', 0.9545455, 'accuracy(val)', 0.96053332)
('step', 2, 'loss(train)', 0.021942213, 'loss(val)', 0.12042106, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96533334)
('step', 3, 'loss(train)', 0.022465935, 'loss(val)', 0.11429107, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96773332)
('step', 4, 'loss(train)', 0.012696098, 'loss(val)', 0.1234107, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96799999)
('step', 5, 'loss(train)', 0.031989589, 'loss(val)', 0.12998861, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96786666)
('step', 6, 'loss(train)', 0.0058744983, 'loss(val)', 0.13374893, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96706671)
('step', 7, 'loss(train)', 0.0061401678, 'loss(val)', 0.14277826, 'accuracy(train)', 1.0, 'accurac

In [75]:
Xtst = np.load("hw2-test-data.npy")
yTst = stoch(batch = 22, learning_rate =  0.047894, epochs = 25, momentum = 0.0, only_last = False, optimizer = "None", test = Xtst)
yTst = np.argmax(yTst,axis=1)
np.savetxt('hw2-test-labels_1.txt',yTst,fmt='%d')

('batch =', 22, 'learning_rate =', 0.047894, 'momentum = ', 0.0)
('step', 0, 'loss(train)', 0.13021183, 'loss(val)', 0.20379642, 'accuracy(train)', 0.9545455, 'accuracy(val)', 0.93760002)
('step', 1, 'loss(train)', 0.12701891, 'loss(val)', 0.14979909, 'accuracy(train)', 0.9545455, 'accuracy(val)', 0.95573336)
('step', 2, 'loss(train)', 0.089834936, 'loss(val)', 0.12967969, 'accuracy(train)', 0.9545455, 'accuracy(val)', 0.96026665)
('step', 3, 'loss(train)', 0.029468458, 'loss(val)', 0.12416247, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96466666)
('step', 4, 'loss(train)', 0.019837365, 'loss(val)', 0.12280801, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96613336)
('step', 5, 'loss(train)', 0.017575169, 'loss(val)', 0.1284844, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96719998)
('step', 6, 'loss(train)', 0.0055899238, 'loss(val)', 0.12558277, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96920002)
('step', 7, 'loss(train)', 0.0077787885, 'loss(val)', 0.13359386, 'accuracy(train)', 1.0, 'ac

In [76]:
#) This model is chosen as the best

Xtst = np.load("hw2-test-data.npy")
yTst = stoch(batch = 22, learning_rate =  0.047894, epochs = 40, momentum = 0.0, only_last = False, optimizer = "None", test = Xtst)
yTst = np.argmax(yTst,axis=1)
np.savetxt('hw2-test-labels_2.txt',yTst,fmt='%d')

('batch =', 22, 'learning_rate =', 0.047894, 'momentum = ', 0.0)
('step', 0, 'loss(train)', 0.1756589, 'loss(val)', 0.19645184, 'accuracy(train)', 0.9545455, 'accuracy(val)', 0.94440001)
('step', 1, 'loss(train)', 0.051821887, 'loss(val)', 0.14126152, 'accuracy(train)', 0.9545455, 'accuracy(val)', 0.9612)
('step', 2, 'loss(train)', 0.010198888, 'loss(val)', 0.12489749, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96680003)
('step', 3, 'loss(train)', 0.0030813701, 'loss(val)', 0.12065139, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96853334)
('step', 4, 'loss(train)', 0.001447467, 'loss(val)', 0.13077128, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96746665)
('step', 5, 'loss(train)', 0.0045563728, 'loss(val)', 0.12670931, 'accuracy(train)', 1.0, 'accuracy(val)', 0.97079998)
('step', 6, 'loss(train)', 0.00089044712, 'loss(val)', 0.13525937, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96946669)
('step', 7, 'loss(train)', 0.00064938265, 'loss(val)', 0.13717698, 'accuracy(train)', 1.0, 'accurac